In [11]:

import tkinter as tk
from tkinter import simpledialog, messagebox
import csv
import pandas as pd

In [12]:
# Lectura de archivos y creación de diccionarios

with open("./Español.txt", "r", encoding="utf-8") as archivo1:
    contenido_archivo1 = archivo1.read().splitlines()
with open("./Ingles.txt", "r", encoding="utf-8") as archivo2:
    contenido_archivo2 = archivo2.read().splitlines()

diccionario_espanol_ingles = {palabra.lower(): traduccion.lower() for palabra, traduccion in zip(contenido_archivo1, contenido_archivo2)}
diccionario_ingles_espanol = {palabra.lower(): traduccion.lower() for palabra, traduccion in zip(contenido_archivo2, contenido_archivo1)}


In [13]:
root = tk.Tk()
root.title("Traductor")
root.geometry("350x200")

palabra_label = tk.Label(root, text="Oracion 1:")
palabra2_label = tk.Label(root, text="Oracion 2:")
palabra_entry = tk.Entry(root)
palabra2_entry = tk.Entry(root)
resultado_label = tk.Label(root, text="")

palabra_label.place(x=80, y=30)
palabra2_label.place(x=80, y=50)
palabra_entry.place(x=140, y=30)
palabra2_entry.place(x=140, y=50)
resultado_label.place(x=40, y=140)



In [14]:
def agregar_etiqueta(palabra, idioma_origen, etiquetas):


    if idioma_origen == 'español':
        palabra_traducida = simpledialog.askstring("Entrada", f"Por favor, introduce la traducción en inglés para '{palabra}':")
        gramatica = simpledialog.askstring("Entrada", f"Por favor, introduce la etiqueta gramatical para '{palabra}':")
        etiquetas[palabra] = (palabra_traducida, gramatica)
        etiquetas[palabra_traducida] = (palabra, gramatica)
    else:
        palabra_traducida = simpledialog.askstring("Entrada", f"Por favor, introduce la traducción en español para '{palabra}':")
        gramatica = simpledialog.askstring("Entrada", f"Por favor, introduce la etiqueta gramatical para '{palabra}':")
        etiquetas[palabra] = (palabra_traducida, gramatica)
        etiquetas[palabra_traducida] = (palabra, gramatica)

    with open('etiquetas.csv', 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([palabra, palabra_traducida, gramatica])

    return palabra_traducida, gramatica

In [15]:
def buscar_regla_traducida(etiquetas_origen, idioma_origen):
    
    with open('resultados.csv', mode='r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        for row in reader:
            
            if idioma_origen == 'español':  
                if row[0].strip().lower() == etiquetas_origen.strip().lower():
                    return row[1] 
                
            elif idioma_origen == 'ingles':
                if row[1].strip().lower() == etiquetas_origen.strip().lower():
                    return row[0]
                    
   

In [16]:
etiquetas_dict = {}
with open('etiquetas.csv', mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        etiquetas_dict[row['Español'].lower()] = {'traduccion': row['Ingles'], 'etiqueta': row['Etiqueta']}
        etiquetas_dict[row['Ingles'].lower()] = {'traduccion': row['Español'], 'etiqueta': row['Etiqueta']}



In [17]:


# Función para etiquetar una oración
def etiquetar_oracion(oracion, etiquetas_dict):
    palabras = oracion.split()
    etiquetas = [etiquetas_dict.get(palabra.lower(), {'etiqueta': 'desconocido'})['etiqueta'] for palabra in palabras]
    return ' '.join(etiquetas)

# Función principal para procesar la oración ingresada
def procesar_oracion(oracion, etiquetas_dict):
    etiquetas = etiquetar_oracion(oracion, etiquetas_dict)
    return etiquetas
        

In [18]:
# Función para construir la oración en el idioma destino

def construir_oracion(oracion_origen, etiquetas_origen, etiquetas_destino, etiquetas_dict):
    
    palabras_origen = oracion_origen.split()
    etiquetas_origen_list = etiquetas_origen.split()
    oracion_destino = []

    for etiqueta in etiquetas_destino.split():
        for i, etiqueta_origen in enumerate(etiquetas_origen_list):
            if etiqueta_origen == etiqueta:
                palabra_destino = etiquetas_dict.get(palabras_origen[i].lower(), {'traduccion': palabras_origen[i]})['traduccion']
                oracion_destino.append(palabra_destino)
                break

    return ' '.join(oracion_destino)

In [19]:
# Función principal
def traducir_palabra():

    oracion = palabra_entry.get().lower()
    oracion2 = palabra2_entry.get().lower()

    palabras = oracion.split()
    palabras2 = oracion2.split()

    primera_palabra = palabras[0]

    if primera_palabra in diccionario_espanol_ingles:
        traduccion = diccionario_espanol_ingles[primera_palabra]
        idioma_origen = 'español'
    elif primera_palabra in diccionario_ingles_espanol:
        traduccion = diccionario_ingles_espanol[primera_palabra]
        idioma_origen = 'ingles'

   
        
    print(f'Idioma Origen : {idioma_origen}')
    etiquetas_origen = procesar_oracion(oracion, etiquetas_dict)

    if not oracion2:
        regla_gramatical = buscar_regla_traducida(etiquetas_origen, idioma_origen)
    else:
        regla_gramatical = procesar_oracion(oracion2, etiquetas_dict)
        with open('resultados.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([etiquetas_origen, regla_gramatical])

    print(f'Regla Original: {etiquetas_origen} \nRegla traducida: {regla_gramatical}\n\n')
    
   



    oracion_destino = construir_oracion(oracion, etiquetas_origen, regla_gramatical, etiquetas_dict)
    resultado_label.config(text=f"Traducción: {oracion_destino}.")




In [20]:

traducir_button = tk.Button(root, text="Traducir", command=traducir_palabra)
traducir_button.place(x=130, y=90)

root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\diego\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\diego\AppData\Local\Temp\ipykernel_356\1236804631.py", line 21, in traducir_palabra
    print(f'Idioma Origen : {idioma_origen}')
                             ^^^^^^^^^^^^^
UnboundLocalError: cannot access local variable 'idioma_origen' where it is not associated with a value
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\diego\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\diego\AppData\Local\Temp\ipykernel_356\1236804631.py", line 21, in traducir_palabra
    print(f'Idioma Origen : {idioma_origen}')
                             ^^^^^^^^^^^^^
UnboundLocalError: cannot access local variable 'i